# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nikolskoye,59.7035,30.7861,52.14,88,7,6.38,RU,1600117200
1,1,Los Llanos de Aridane,28.6585,-17.9182,71.60,79,69,9.84,ES,1600117200
2,2,Mataura,-46.1927,168.8643,43.56,86,57,6.17,NZ,1600117200
3,3,Asău,46.4333,26.4000,61.23,84,1,3.49,RO,1600117200
4,4,Tautira,-17.7333,-149.1500,80.47,70,10,10.20,PF,1600117200


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Lat", "Lng"]]

In [17]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
edited_city_df = weather_df.loc[(cities["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
edited_city_df = edited_city_df.dropna(how='any')
edited_city_df.reset_index(inplace=True)
del edited_city_df['index']
edited_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,16,Rikitea,-23.1203,-134.9692,73.72,81,0,11.65,PF,1600117200
1,19,Moberly,39.4184,-92.4382,77.58,63,0,7.05,US,1600117200
2,55,Port Keats,-14.2500,129.5500,73.47,88,0,7.83,AU,1600117200
3,160,Arraial do Cabo,-22.9661,-42.0278,77.38,74,0,24.94,BR,1600117200
4,187,São João da Barra,-21.6403,-41.0511,77.22,68,0,21.77,BR,1600117200


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotellist = []

for i in range(len(edited_city_df)):
    lat = edited_city_df.loc[i]['Lat']
    lng = edited_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
edited_city_df["Hotel Name"] = hotellist
hotel_df = edited_city_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,16,Rikitea,-23.1203,-134.9692,73.72,81,0,11.65,PF,1600117200,Rikitea
1,19,Moberly,39.4184,-92.4382,77.58,63,0,7.05,US,1600117200,Moberly
2,55,Port Keats,-14.2500,129.5500,73.47,88,0,7.83,AU,1600117200,Thamarrurr
3,160,Arraial do Cabo,-22.9661,-42.0278,77.38,74,0,24.94,BR,1600117200,Cabo Frio
4,187,São João da Barra,-21.6403,-41.0511,77.22,68,0,21.77,BR,1600117200,Grussaí


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))